# Medication Adherence Risk – BI & Dashboard Prep Notebook

This notebook prepares aggregated datasets for use in **Power BI** / **Tableau** dashboards.

It assumes you have already:
- Generated synthetic raw data in `data/raw/`
- Built the training dataset in `data/processed/training_dataset.csv`
- Trained the model and saved it as `models/rf_best_model.joblib`
- (Optionally) created `data/processed/outputs/scored_members.csv`


## 1. Setup & Paths

In [6]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import joblib

BASE_DIR = Path.cwd()
RAW_DIR = BASE_DIR / "data" / "raw"
PROC_DIR = BASE_DIR / "data" / "processed"
OUT_DIR = PROC_DIR / "bi"
MODEL_DIR = BASE_DIR / "models"

OUT_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_FILE = PROC_DIR / "training_dataset.csv"
SCORED_FILE = PROC_DIR / "outputs" / "scored_members.csv"
MODEL_PATH = MODEL_DIR / "rf_best_model.joblib"

print("BASE_DIR:", TRAIN_FILE)
print("TRAIN_FILE exists:", TRAIN_FILE.exists())
print("SCORED_FILE exists:", SCORED_FILE.exists())
print("MODEL_PATH exists:", MODEL_PATH.exists())

BASE_DIR: /Users/ganeshkrishnalakshmisetty/med-adherence-risk/notebooks/data/processed/training_dataset.csv
TRAIN_FILE exists: False
SCORED_FILE exists: False
MODEL_PATH exists: False


## 2. Load or Create Scored Members Dataset

We try to load `scored_members.csv` (batch-scored dataset). If it doesn't exist, we:
1. Load `training_dataset.csv`
2. Load the trained pipeline model
3. Compute `prob_non_adherent` and `risk_category` for each row
4. Save `scored_members.csv` for future reuse


In [4]:
def assign_risk_category(prob_nonadherent: float) -> str:
    if prob_nonadherent >= 0.8:
        return "HIGH"
    elif prob_nonadherent >= 0.6:
        return "MEDIUM"
    else:
        return "LOW"

if SCORED_FILE.exists():
    df_scored = pd.read_csv(SCORED_FILE, parse_dates=["start_date"])
    print("Loaded existing scored_members.csv:", df_scored.shape)
else:
    print("scored_members.csv not found; creating it from training data and model...")
    df_train = pd.read_csv(TRAIN_FILE, parse_dates=["start_date"])
    model = joblib.load(MODEL_PATH)

    DROP_COLS = [
        "label_non_adherent",
        "pdc_180d",
    ]

    X = df_train.drop(columns=DROP_COLS, errors="ignore")
    proba = model.predict_proba(X)[:, 1]

    df_scored = df_train.copy()
    df_scored["prob_non_adherent"] = proba
    df_scored["risk_category"] = df_scored["prob_non_adherent"].apply(assign_risk_category)

    output_dir = SCORED_FILE.parent
    output_dir.mkdir(parents=True, exist_ok=True)
    df_scored.to_csv(SCORED_FILE, index=False)
    print("Created and saved scored_members.csv:", df_scored.shape)

df_scored.head()

scored_members.csv not found; creating it from training data and model...


FileNotFoundError: [Errno 2] No such file or directory: '/Users/ganeshkrishnalakshmisetty/med-adherence-risk/notebooks/data/processed/training_dataset.csv'

## 3. KPI Calculations

We compute high-level KPIs you can surface as tiles in Power BI / Tableau:
- **% High-Risk Members**
- **Average Non-Adherence Risk by Drug Class**


In [ ]:
# Overall % high-risk members
total_members = len(df_scored)
high_risk_members = (df_scored["risk_category"] == "HIGH").sum()
pct_high_risk = high_risk_members / total_members if total_members > 0 else 0

print(f"Total members: {total_members}")
print(f"High-risk members: {high_risk_members}")
print(f"% High-Risk Members: {pct_high_risk:.2%}")

# Average non-adherence risk by drug class
if "drug_class" in df_scored.columns:
    kpi_drug = (
        df_scored
        .groupby("drug_class")
        ["prob_non_adherent"]
        .agg(["count", "mean"])
        .rename(columns={"count": "member_count", "mean": "avg_prob_non_adherent"})
        .sort_values("avg_prob_non_adherent", ascending=False)
    )
    try:
        display(kpi_drug)
    except NameError:
        print(kpi_drug.head())
    kpi_drug.to_csv(OUT_DIR / "kpi_by_drug_class.csv")
else:
    print("Column 'drug_class' not found; cannot compute KPI by drug class.")

## 4. Geo-Level Aggregations (County / State)

We prepare a dataset suitable for **map visuals**:
- Number of high-risk members by county/state
- High-risk rate
- Overlay SDOH metrics (if present in the scored data)


In [ ]:
geo_cols = ["county", "state"]
for col in geo_cols:
    if col not in df_scored.columns:
        print(f"Column '{col}' missing; geo aggregations may be incomplete.")

df_geo = df_scored.copy()
df_geo["is_high_risk"] = (df_geo["risk_category"] == "HIGH").astype(int)

group_cols = [c for c in ["county", "state"] if c in df_geo.columns]
if group_cols:
    agg_dict = {
        "is_high_risk": ["sum", "count"],
    }

    # Include SDOH metrics if available
    sdoh_cols = [
        "pct_uninsured", "pct_food_stamp", "pct_public_transport",
        "pct_less_hs_edu", "pct_disabled", "total_mh_providers"
    ]
    for col in sdoh_cols:
        if col in df_geo.columns:
            agg_dict[col] = "mean"

    df_geo_agg = (
        df_geo
        .groupby(group_cols)
        .agg(agg_dict)
    )

    # Flatten MultiIndex columns
    df_geo_agg.columns = ["_".join([c for c in col if c]) for col in df_geo_agg.columns.values]
    df_geo_agg = df_geo_agg.reset_index()

    # Rename high-risk cols
    df_geo_agg = df_geo_agg.rename(columns={
        "is_high_risk_sum": "high_risk_count",
        "is_high_risk_count": "total_members"
    })
    df_geo_agg["high_risk_rate"] = (
        df_geo_agg["high_risk_count"] / df_geo_agg["total_members"]
    )

    try:
        display(df_geo_agg.head())
    except NameError:
        print(df_geo_agg.head())
    df_geo_agg.to_csv(OUT_DIR / "geo_high_risk_by_county_state.csv", index=False)
else:
    print("No geo columns available; skipping geo aggregation.")

## 5. Trend Aggregations Over Time

We aggregate risk over time to support **trend charts** in BI tools:
- High-risk count by month
- Overall member count by month


In [ ]:
if "start_date" in df_scored.columns:
    df_trend = df_scored.copy()
    df_trend["start_month"] = df_trend["start_date"].dt.to_period("M").dt.to_timestamp()
    df_trend["is_high_risk"] = (df_trend["risk_category"] == "HIGH").astype(int)

    df_trend_agg = (
        df_trend
        .groupby("start_month")
        .agg(
            total_members=("member_id", "count"),
            high_risk_count=("is_high_risk", "sum"),
            avg_prob_non_adherent=("prob_non_adherent", "mean")
        )
        .reset_index()
    )

    df_trend_agg["high_risk_rate"] = (
        df_trend_agg["high_risk_count"] / df_trend_agg["total_members"]
    )

    try:
        display(df_trend_agg.head())
    except NameError:
        print(df_trend_agg.head())
    df_trend_agg.to_csv(OUT_DIR / "trend_high_risk_by_month.csv", index=False)
else:
    print("Column 'start_date' missing; cannot compute time trends.")

## 6. High-Risk by Top Pharmacies / Providers

We join scored members with **primary pharmacy** and **primary provider**
derived from `pharmacy_claims.csv`, and then aggregate:
- High-risk count by pharmacy
- High-risk count by ordering provider

We define a member's primary pharmacy/provider as the one with the highest
number of claims in their history.


In [ ]:
pharm_file = RAW_DIR / "pharmacy_claims.csv"
if pharm_file.exists():
    df_claims = pd.read_csv(pharm_file, parse_dates=["fill_date"])

    # Compute primary pharmacy per member
    grp_ph = (
        df_claims
        .groupby(["member_id", "pharmacy_id"])
        .size()
        .reset_index(name="claim_count")
    )
    # For each member, take pharmacy_id with max claim_count
    idx_max_ph = grp_ph.groupby("member_id")["claim_count"].idxmax()
    df_primary_pharm = grp_ph.loc[idx_max_ph, ["member_id", "pharmacy_id"]]

    # Compute primary provider per member
    grp_pr = (
        df_claims
        .groupby(["member_id", "ordering_provider_id"])
        .size()
        .reset_index(name="claim_count")
    )
    idx_max_pr = grp_pr.groupby("member_id")["claim_count"].idxmax()
    df_primary_prov = grp_pr.loc[idx_max_pr, ["member_id", "ordering_provider_id"]]

    # Join with scored members
    df_scored_ph = df_scored.merge(df_primary_pharm, on="member_id", how="left")
    df_scored_ph_pr = df_scored_ph.merge(df_primary_prov, on="member_id", how="left")
    df_scored_ph_pr["is_high_risk"] = (df_scored_ph_pr["risk_category"] == "HIGH").astype(int)

    # Top 10 pharmacies by member volume
    df_pharm_agg = (
        df_scored_ph_pr
        .groupby("pharmacy_id")
        .agg(
            total_members=("member_id", "count"),
            high_risk_count=("is_high_risk", "sum"),
            avg_prob_non_adherent=("prob_non_adherent", "mean")
        )
        .reset_index()
        .sort_values("total_members", ascending=False)
    )
    df_pharm_agg["high_risk_rate"] = (
        df_pharm_agg["high_risk_count"] / df_pharm_agg["total_members"]
    )

    df_top_pharm = df_pharm_agg.head(10)
    try:
        display(df_top_pharm)
    except NameError:
        print(df_top_pharm.head())
    df_top_pharm.to_csv(OUT_DIR / "high_risk_by_top_pharmacies.csv", index=False)

    # Top 10 providers by member volume
    df_prov_agg = (
        df_scored_ph_pr
        .groupby("ordering_provider_id")
        .agg(
            total_members=("member_id", "count"),
            high_risk_count=("is_high_risk", "sum"),
            avg_prob_non_adherent=("prob_non_adherent", "mean")
        )
        .reset_index()
        .sort_values("total_members", ascending=False)
    )
    df_prov_agg["high_risk_rate"] = (
        df_prov_agg["high_risk_count"] / df_prov_agg["total_members"]
    )

    df_top_prov = df_prov_agg.head(10)
    try:
        display(df_top_prov)
    except NameError:
        print(df_top_prov.head())
    df_top_prov.to_csv(OUT_DIR / "high_risk_by_top_providers.csv", index=False)
else:
    print("pharmacy_claims.csv not found; skipping top pharmacies/providers aggregations.")

## 7. Summary

This notebook generated aggregated datasets under `data/processed/bi/` that can be used in Power BI / Tableau:

- `kpi_by_drug_class.csv`
- `geo_high_risk_by_county_state.csv`
- `trend_high_risk_by_month.csv`
- `high_risk_by_top_pharmacies.csv`
- `high_risk_by_top_providers.csv`

In BI, you can build:
- KPI tiles: overall % high-risk, avg risk by drug class
- Maps: high-risk count & rate by county/state, with SDOH overlays
- Trend charts: high-risk count & rate over time, by month
- Bar charts: high-risk rate by top pharmacies and providers
